# Binomial Model convergence to Black-Scoles

In [10]:
using CSV
using DelimitedFiles, DataFrames
using Statistics
#import PyPlot as plt
#using Plots
using GLM
using ShiftedArrays
using PyCall
using PlotlyJS
using FinancialToolbox
using FinancialDerivatives
using RCall

@pyinclude("Leisen_Reimer.py")

In [2]:
JNJ = DataFrame(CSV.File("DATA/JNJ.csv"))

,Date,Open,High,Low,Close,Adj Close,Volume
,Date,Float64,Float64,Float64,Float64,Float64,Int64
1,2017-03-21,128.38,128.45,127.14,127.25,111.545,7578600
2,2017-03-22,127.05,127.65,126.21,126.26,110.677,10272700
3,2017-03-23,126.1,127.0,125.66,125.9,110.361,8508100
4,2017-03-24,125.86,126.36,125.13,125.48,109.993,6490300
5,2017-03-27,125.16,126.14,125.15,125.8,110.274,5004300
6,2017-03-28,125.62,125.78,124.78,125.66,110.151,5287600
7,2017-03-29,125.05,125.33,124.36,124.92,109.502,3971800
8,2017-03-30,124.74,125.43,124.28,124.66,109.274,4711700
9,2017-03-31,124.11,124.83,124.03,124.55,109.178,5657600


In [3]:
p = PlotlyJS.plot(candlestick(x    =JNJ[!, "Date" ],
                              open =JNJ[!, "Open" ],
                              high =JNJ[!, "High" ],
                              low  =JNJ[!, "Low"  ],
                              close=JNJ[!, "Close"] ),
                  Layout(title="JNJ stock Prices",
                         yaxis_title="JNJ Stock")
    )

data: [
  "candlestick with fields close, high, low, open, type, and x"
]

layout: "layout with fields margin, template, title, and yaxis"

In [4]:
CALL_1MONTH   = DataFrame(CSV.File("DATA/CALL_1MONTH.csv"  ))
CALL_3MONTHS  = DataFrame(CSV.File("DATA/CALL_3MONTHS.csv" ))
CALL_7MONTHS  = DataFrame(CSV.File("DATA/CALL_7MONTHS.csv" ))
CALL_10MONTHS = DataFrame(CSV.File("DATA/CALL_10MONTHS.csv"))
CALL_15MONTHS = DataFrame(CSV.File("DATA/CALL_15MONTHS.csv"))
CALL_22MONTHS = DataFrame(CSV.File("DATA/CALL_22MONTHS.csv"))

PUT_1MONTH    = DataFrame(CSV.File("DATA/PUT_1MONTH.csv"   ))
PUT_3MONTHS   = DataFrame(CSV.File("DATA/PUT_3MONTHS.csv"  ))
PUT_7MONTHS   = DataFrame(CSV.File("DATA/PUT_7MONTHS.csv"  ))
PUT_10MONTHS  = DataFrame(CSV.File("DATA/PUT_10MONTHS.csv" ))
PUT_15MONTHS  = DataFrame(CSV.File("DATA/PUT_15MONTHS.csv" ))
PUT_22MONTHS  = DataFrame(CSV.File("DATA/PUT_22MONTHS.csv" ))

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask
,String,String,Float64,Float64,Float64,Float64
1,JNJ240119P00080000,2022-03-15 9:32AM EDT,80.0,1.25,0.95,2.02
2,JNJ240119P00085000,2022-03-07 3:51PM EDT,85.0,1.6,1.2,1.95
3,JNJ240119P00090000,2022-03-04 2:54PM EDT,90.0,1.98,1.5,2.52
4,JNJ240119P00095000,2022-03-10 2:06PM EDT,95.0,2.29,1.5,2.51
5,JNJ240119P00100000,2022-03-21 12:22PM EDT,100.0,1.7,1.46,2.34
6,JNJ240119P00105000,2022-03-17 3:29PM EDT,105.0,2.5,1.91,2.57
7,JNJ240119P00110000,2022-03-16 10:29AM EDT,110.0,2.91,1.96,3.1
8,JNJ240119P00115000,2022-03-21 10:46AM EDT,115.0,3.3,2.26,3.45
9,JNJ240119P00120000,2022-03-15 10:48AM EDT,120.0,3.71,3.25,4.05


In [5]:
#INTEREST_RATES = Dict("overnight"=>.0032871,
#                      "1 month"  =>.0044857,
#                      "3 month"  =>.0093400,
#                      "6 month"  =>.0128757,
#                      "12 month" =>.0178643)
INTEREST_RATES = [.0044857, .0093400, .0128757, .0178643, .0178643, .0178643]
S0         = JNJ.Close[length(JNJ.Close)]
Maturity1  = 1  / 12
Maturity3  = 3  / 12
Maturity7  = 7  / 12
Maturity10 = 10 / 12
Maturity15 = 15 / 12
Maturity22 = 22 / 12
Maturities = [1,3,7,10,15,22]

Strike     = Int(round(S0))
Year_Days  = 252
#CALL_1MONTH[CALL_1MONTH.Strike.==Int(round(S0)),"Last Price"][1]

252

In [6]:
function Return( df )
    df[!,:Return    ] =     ( df.Close - lag(df.Close, 1) ) ./ lag(df.Close, 1)
    df[!,:Return_LOG] = log.( df.Close                      ./ lag(df.Close, 1) )
end

function BINOMIAL_EU(S, K, T, r, σ, N, type_ = "call")
    Δt     = T / N
    U      = exp(σ * √Δt)
    D      = 1 / U
    #R      = ( 1 + r * Δt )
    R      = exp(r * Δt)
    q      = (R - D) / (U - D)
    
    value = 0 

    for i = 1:N+1
        node_prob = binomial(BigInt(N),BigInt(i)) * q^i * (1-q)^(N-i)
        ST        = S * U^i * D^(N-i)
        if type_ == "call"
            value += max(ST-K, 0) * node_prob
        elseif type_ == "put"
            value += max(K-ST, 0) * node_prob
        else
            throw(ValueError("type_ must be 'call' or 'put'"))
        end
    end
    return value * exp( -r * T )
end

BINOMIAL_EU (generic function with 2 methods)

In [7]:
import FinancialDerivatives.h, FinancialDerivatives.evaluate, FinancialDerivatives.Option

function h(z::T, n::Int64) where {T<:Number}
    return 0.5 + sign(z) * 0.5 * sqrt(1 - exp(-((z / (n + 1.0 / 3.0 +.1/(n+1)))^2.0) * (n + 1.0 / 6.0)))
end

function evaluate(O::Option, m::LeisenReimer, N::Int64 = 1001)
    
    if (N%2 == 1)
        N = N
    else
        N = N+1
    end
    
    Δt = O.t / N
    R  = exp(O.r * Δt)
    d1 = (log(O.s / O.k) + (O.r + O.σ * O.σ / 2) * O.t) / (O.σ * √O.t)
    d2 = d1 - O.σ * √O.t
    p  = h(d2, N)
    q  = 1 - p

    if O.call == -1
        Z = [max(0, O.k - O.s * exp((2 * i - N) * O.σ * √Δt)) for i = 0:N]
    elseif O.call == 1
        Z = [max(0, O.s * exp((2 * i - N) * O.σ * √Δt) - O.k) for i = 0:N]
    end
    
    for n = N-1:-1:0, i = 0:n
        if O.call == -1
            x = O.k - O.s * exp((2 * i - n) * O.σ * √Δt)
        elseif O.call == 1
            x = O.s * exp((2 * i - n) * O.σ * √Δt) - O.k
        end
        y = (q * Z[i+1] + p * Z[i+2]) / exp(O.r * Δt)
        Z[i+1] = max(x, y)
    end
    
    return Z[1]
end

evaluate (generic function with 28 methods)

In [19]:
Return( JNJ )

function Volatility( Returns, T, daily = false )
    VOL_DAILY  = std(skipmissing(Returns[length(Returns)-T:length(Returns)]))
    VOL_ANNUAL = VOL_DAILY * sqrt(Year_Days)
    if daily==false
        return VOL_ANNUAL
    end
    return VOL_DAILY, VOL_ANNUAL
end

VOL_ANNUAL = [Volatility( JNJ.Return, i*30 ) for i in Maturities]

REAL_CALL = [CALL_1MONTH[    CALL_1MONTH.Strike.==Int(round(S0)),"Last Price"][1],
             CALL_3MONTHS[  CALL_3MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
             CALL_7MONTHS[  CALL_7MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
             CALL_10MONTHS[CALL_10MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
             CALL_15MONTHS[CALL_15MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
             CALL_22MONTHS[CALL_22MONTHS.Strike.==Int(round(S0)),"Last Price"][1]]

BINOMIAL      = Array{Float64}( undef, length(Maturities), 100 )
LEISEN_REIMER = Array{Float64}( undef, length(Maturities), 50  )
BLACK_SCHOLES = Array{Float64}( undef, length(Maturities) )

for i in 1:length(Maturities)
    EuroCall = EuropeanOption(Float64(S0), Float64(Strike), Float64(INTEREST_RATES[i]), Float64(VOL_ANNUAL[i]), Float64(Maturities[i])/12., 1)
    BINOMIAL[i,:]      = [evaluate(EuroCall, CoxRossRubinstein(), j) for j in 1:1:100]
    #LEISEN_REIMER[i,:] = [LeisenReimer_(S0, Strike, Maturities[i], INTEREST_RATES[i], VOL_ANNUAL[i], j)    for j in 1:2:100]
    #LEISEN_REIMER[i,:] = [evaluate(EuroCall, LeisenReimer(), j)      for j in 1:2:100]
    LEISEN_REIMER[i,:] = [py"LeisenReimerBinomial"("P", "e", "C", S0, Strike, Maturities[i]/12, INTEREST_RATES[i], INTEREST_RATES[i], VOL_ANNUAL[i], j) for j in 1:2:100]
    BLACK_SCHOLES[i]   =  evaluate(EuroCall, BlackScholes())        
end

In [22]:
function Plot_BIN_BL( BINOMIAL, BL, LR, REAL, T )
    BIN_PLOT  = PlotlyJS.scatter(;x=1:length(BINOMIAL),   y=BINOMIAL,mode="markers+lines", name="Binomial Model", line=attr(color="blue", width=2, dash="dash"  ))
    BL_PLOT   = PlotlyJS.scatter(;x=1:length(BINOMIAL),   y=repeat([BL], length(BINOMIAL)), mode="lines", name="Black-Scholes prediction", line=attr(color="black", width=4, dash="dashdot"))
    LR_PLOT   = PlotlyJS.scatter(;x=1:2:length(BINOMIAL), y=LR,mode="markers+lines", name="Leisen-Reimer", line=attr(color="red", width=3, dash="dot" ))
    #REAL_PLOT = PlotlyJS.scatter(;x=1:length(BINOMIAL), y=repeat([REAL], length(BINOMIAL)), mode="lines", name="Real value", line=attr(color="firebrick", width=2, dash="dash"))
    layout    = Layout(title=string("Johnson&Johnson Call Price, Maturity ", Int(T), " months, Strike ~ S0, as function of number of steps"),
                       yaxis_title="CALL price [\$]", xaxis_title="Steps")
    #return PlotlyJS.plot([BIN_PLOT,BL_PLOT, REAL_PLOT], layout)
    return PlotlyJS.plot([BIN_PLOT,LR_PLOT,BL_PLOT], layout)
end

p = Plot_BIN_BL( BINOMIAL[1,:], BLACK_SCHOLES[1], LEISEN_REIMER[1,:], REAL_CALL[1], Maturities[1] )

data: [
  "scatter with fields line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [23]:
p = Plot_BIN_BL( BINOMIAL[2,:], BLACK_SCHOLES[2], LEISEN_REIMER[2,:], REAL_CALL[1], Maturities[2] )

data: [
  "scatter with fields line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [27]:
p = PlotlyJS.make_subplots(rows=3, cols=2, shared_xaxes=true, vertical_spacing=0.08, y_title="CALL price [\$]", x_title="Steps", subplot_titles=["1 Months maturity" "7 Months maturity" "15 Months maturity"; "3 Months maturity" "10 Months maturity" "22 Months maturity"])

r = c = 1
for i in 1:length(Maturities)
    PlotlyJS.add_trace!(p, PlotlyJS.scatter(;x=1:length(BINOMIAL[i,:]), y=BINOMIAL[i,:],mode="markers+lines", line=attr(color="blue", width=2)), row=r, col=c)
    PlotlyJS.add_trace!(p, PlotlyJS.scatter(;x=1:length(BINOMIAL[i,:]), y=repeat([BLACK_SCHOLES[i]], length(BINOMIAL[i,:])), mode="lines", line=attr(color="black", width=3, dash="dashdot")), row=r, col=c)
    PlotlyJS.add_trace!(p, PlotlyJS.scatter(;x=1:2:length(BINOMIAL), y=LEISEN_REIMER[i,:],mode="markers+lines", name="Leisen-Reimer", line=attr(color="red", width=3, dash="dot" )), row=r, col=c)
    PlotlyJS.add_trace!(p, PlotlyJS.scatter(;x=1:length(BINOMIAL[i,:]), y=repeat([REAL_CALL[i]], length(BINOMIAL[i,:])), mode="lines", name="Real value", line=attr(color="firebrick", width=2, dash="dash")), row=r, col=c)
    c+=1
    if c == 3
        c  = 1
        r += 1
    end
end
    
p.plot.layout.annotations[7].yshift=-15
p.plot.layout.annotations[8].xshift=-18
PlotlyJS.relayout!(p, title_text="Johnson&Johnson Call Price, Strike ~ S0, as function of number of steps for several maturities", showlegend=false, width=1000, height=1200)
#print(json(p.plot.layout.annotations, 2))
p

data: [
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, height, margin, showlegend, template, title, width, xaxis, xaxis2, xaxis3, xaxis4, xaxis5, xaxis6, yaxis, yaxis2, yaxis3, yaxis4, yaxis5, and yaxis6"

In [28]:
p = PlotlyJS.make_subplots(rows=2, cols=3, shared_xaxes=true, vertical_spacing=0.08, y_title="CALL price [\$]", x_title="Steps", subplot_titles=["1 Months maturity" "7 Months maturity" "15 Months maturity"; "3 Months maturity" "10 Months maturity" "22 Months maturity"])

r = c = 1
for i in 1:length(Maturities)
    PlotlyJS.add_trace!(p, PlotlyJS.scatter(;x=1:length(BINOMIAL[i,:]), y=BINOMIAL[i,:],mode="markers+lines", line=attr(color="blue", width=2)), row=r, col=c)
    PlotlyJS.add_trace!(p, PlotlyJS.scatter(;x=1:length(BINOMIAL[i,:]), y=repeat([BLACK_SCHOLES[i]], length(BINOMIAL[i,:])), mode="lines", line=attr(color="black", width=3, dash="dashdot")), row=r, col=c)
    PlotlyJS.add_trace!(p, PlotlyJS.scatter(;x=1:2:length(BINOMIAL), y=LEISEN_REIMER[i,:],mode="markers+lines", name="Leisen-Reimer", line=attr(color="red", width=3, dash="dot" )), row=r, col=c)
    PlotlyJS.add_trace!(p, PlotlyJS.scatter(;x=1:length(BINOMIAL[i,:]), y=repeat([REAL_CALL[i]], length(BINOMIAL[i,:])), mode="lines", name="Real value", line=attr(color="firebrick", width=2, dash="dash")), row=r, col=c)
    r+=1
    if r == 3
        r  = 1
        c += 1
    end
end
    
p.plot.layout.annotations[7].yshift=-15
p.plot.layout.annotations[8].xshift=-18
PlotlyJS.relayout!(p, title_text="Johnson&Johnson Call Price, Strike ~ S0, as function of number of steps for several maturities", showlegend=false, width=1000, height=800)
#print(json(p.plot.layout.annotations, 2))
p

data: [
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, height, margin, showlegend, template, title, width, xaxis, xaxis2, xaxis3, xaxis4, xaxis5, xaxis6, yaxis, yaxis2, yaxis3, yaxis4, yaxis5, and yaxis6"

In [30]:
S = K = 100.
r = .01
σ = 0.2
T = 1.
euro_call = EuropeanOption(S, K, r, σ, T, 1)

#bs = blsprice(S, K, r, T, σ)
#bm = [BINOMIAL_EU(S, K, T, r, σ, N) for N in 1:1000]
bs =  evaluate(euro_call, BlackScholes())
bm = [evaluate(euro_call, CoxRossRubinstein(), N) for N in 1:1:500]
#lr = [evaluate(euro_call, LeisenReimer(),      N) for N in 1:2:1000]
lr = [py"LeisenReimerBinomial"("P", "e", "C", S, K, T, r, r, σ, N) for N in 1:2:500]

250-element Vector{Float64}:
 8.265444950977374
 8.403232838109828
 8.42093430274055
 8.426569223379763
 8.429071383442764
 8.430399782905585
 8.431189275248071
 8.431696630815647
 8.432041971526694
 8.43228764143732
 8.432468628853853
 8.432605808162036
 8.432712261804822
 ⋮
 8.433316926612692
 8.433316941293437
 8.433316955778158
 8.433316970089304
 8.433316984218019
 8.433316998181288
 8.433317011962012
 8.43331702560406
 8.43331703905138
 8.43331705233945
 8.433317065463873
 8.433317078461519

In [33]:
Plot_BIN_BL( bm, bs, lr, 10, 12 )

data: [
  "scatter with fields line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"